# Workforce Optimization with cuOpt Python API

This notebook demonstrates how to solve a workforce optimization problem using the cuOpt Python API. The problem involves assigning workers to shifts while minimizing total labor costs.

## Problem Description

We need to assign workers to shifts such that:
- Each shift has the required number of workers.
- Workers can only be assigned to shifts they are available for.
- Total labor cost is minimized.

This is a classic assignment problem that can be formulated as a Mixed Integer Linear Program (MILP).

## Environment Setup

First, let's check if we have a GPU available and install necessary dependencies.


In [1]:
# Check for GPU availability
!nvidia-smi


Tue Sep 30 13:38:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro P620                    On  |   00000000:42:00.0 Off |                  N/A |
| 34%   40C    P8            N/A  /  N/A  |       8MiB /   2048MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Install cuOpt if not already installed
# Uncomment the following line if running in Google Colab or similar environment
# !pip install --upgrade --extra-index-url https://pypi.nvidia.com --user cuopt-cu12 # For cuda 12
# !pip install --upgrade --extra-index-url https://pypi.nvidia.com --user cuopt-cu13 # For cuda 13


## Import Required Libraries


In [3]:
import numpy as np
import pandas as pd
from cuopt.linear_programming.problem import Problem, VType, sense, LinearExpression
from cuopt.linear_programming.solver_settings import SolverSettings
import time


/home/luffy/.local/lib/python3.12/site-packages/cudf/utils/_ptxcompiler.py:64: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/home/luffy/miniforge3/envs/cuopt/lib/python3.12/site-packages/numba_cuda/numba/cuda/cudadrv/driver.py", line 393, in safe_cuda_api_call
    return self._check_cuda_python_error(fname, libfn(*args))
                                                ^^^^^^^^^^^^
TypeError: cuDriverGetVersion() takes no arguments (1 given)


Not patching Numba
  warnings.warn(msg, UserWarning)
/home/luffy/.local/lib/python3.12/site-packages/cupy/_environment.py:596: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run th

## Problem Data Setup

Define the shift requirements, worker pay rates, and availability constraints.


In [4]:
# Number of workers required for each shift
shift_requirements = {
    "Mon1": 3,
    "Tue2": 2,
    "Wed3": 4,
    "Thu4": 2,
    "Fri5": 5,
    "Sat6": 3,
    "Sun7": 4,
    "Mon8": 2,
    "Tue9": 2,
    "Wed10": 3,
    "Thu11": 4,
    "Fri12": 5,
    "Sat13": 7,
    "Sun14": 5,
}

# Amount each worker is paid to work one shift
worker_pay = {
    "Amy": 10,
    "Bob": 12,
    "Cathy": 10,
    "Dan": 8,
    "Ed": 8,
    "Fred": 9,
    "Gu": 11,
}

# Worker availability 
availability = {
    "Amy": ["Tue2", "Wed3", "Fri5", "Sun7", "Tue9", "Wed10", "Thu11", "Fri12", "Sat13", "Sun14"],
    "Bob": ["Mon1", "Tue2", "Fri5", "Sat6", "Mon8", "Thu11", "Sat13", "Sun14"],
    "Cathy": ["Wed3", "Thu4", "Fri5", "Sun7", "Mon8", "Tue9", "Wed10", "Thu11", "Fri12", "Sat13", "Sun14"],
    "Dan": ["Tue2", "Wed3", "Fri5", "Sat6", "Mon8", "Tue9", "Wed10", "Thu11", "Fri12", "Sat13", "Sun14"],
    "Ed": ["Mon1", "Tue2", "Wed3", "Thu4", "Fri5", "Sun7", "Mon8", "Tue9", "Thu11", "Sat13", "Sun14"],
    "Fred": ["Mon1", "Tue2", "Wed3", "Sat6", "Mon8", "Tue9", "Fri12", "Sat13", "Sun14"],
    "Gu": ["Mon1", "Tue2", "Wed3", "Fri5", "Sat6", "Sun7", "Mon8", "Tue9", "Wed10", "Thu11", "Fri12", "Sat13", "Sun14"], 
}

print(f"Number of shifts: {len(shift_requirements)}")
print(f"Number of workers: {len(worker_pay)}")
print(f"Number of available assignments: {sum(len(v) for v in availability.values())}")


Number of shifts: 14
Number of workers: 7
Number of available assignments: 73


In [5]:
# Create DataFrames for better visualization
shifts_df = pd.DataFrame(list(shift_requirements.items()), columns=['Shift', 'Required Workers'])
workers_df = pd.DataFrame(list(worker_pay.items()), columns=['Worker', 'Pay per Shift'])

print("Shift Requirements:")
print(shifts_df)
print("\nWorker Pay Rates:")
print(workers_df)


Shift Requirements:
    Shift  Required Workers
0    Mon1                 3
1    Tue2                 2
2    Wed3                 4
3    Thu4                 2
4    Fri5                 5
5    Sat6                 3
6    Sun7                 4
7    Mon8                 2
8    Tue9                 2
9   Wed10                 3
10  Thu11                 4
11  Fri12                 5
12  Sat13                 7
13  Sun14                 5

Worker Pay Rates:
  Worker  Pay per Shift
0    Amy             10
1    Bob             12
2  Cathy             10
3    Dan              8
4     Ed              8
5   Fred              9
6     Gu             11


## Problem Formulation

Now we'll create the optimization problem using the cuOpt Python API as a MILP. The problem has:
- **Variables**: Binary variables for each (worker, shift) assignment
- **Objective**: Minimize total labor cost
- **Constraints**: Meet shift requirements and respect worker availability


In [6]:
# Create the optimization problem
problem = Problem("workforce_optimization")

# Add binary decision variables for each available (worker, shift) assignment
assignment_vars = {}
for worker, shifts in availability.items():
    for shift in shifts:
        var_name = f"{worker}_{shift}"
        var = problem.addVariable(name=var_name, vtype=VType.INTEGER, lb=0.0, ub=1.0)
        assignment_vars[(worker, shift)] = var

print(f"Created {len(assignment_vars)} binary decision variables")
print(f"Sample variables: {[var.getVariableName() for var in assignment_vars.values()][:5]}")


Created 73 binary decision variables
Sample variables: ['Amy_Tue2', 'Amy_Wed3', 'Amy_Fri5', 'Amy_Sun7', 'Amy_Tue9']


In [7]:
# Create objective function: minimize total labor cost
objective_expr = LinearExpression([], [], 0.0)

for (worker, shift), var in assignment_vars.items():
    cost = worker_pay[worker]
    if cost != 0:  # Only include non-zero coefficients
        objective_expr += var * cost

# Set objective function: minimize total cost
problem.setObjective(objective_expr, sense.MINIMIZE)
print("Objective function set: minimize total labor cost")


Objective function set: minimize total labor cost


In [8]:
# Add constraints: assign exactly the required number of workers to each shift
constraint_names = []

for shift, required_count in shift_requirements.items():
    # Find all workers available for this shift
    shift_assignments = []
    for (worker, shift_name), var in assignment_vars.items():
        if shift_name == shift:
            shift_assignments.append(var)
    
    if len(shift_assignments) > 0:
        # Create constraint: sum of assignments for this shift = required_count
        shift_expr = LinearExpression([], [], 0.0)
        for var in shift_assignments:
            shift_expr += var
        
        constraint = problem.addConstraint(shift_expr == required_count, name=f"shift_{shift}")
        constraint_names.append(f"shift_{shift}")
    else:
        print(f"Warning: No workers available for shift {shift}")

print(f"Added {len(constraint_names)} shift requirement constraints")
print(f"Sample constraints: {constraint_names[:5]}")


Added 14 shift requirement constraints
Sample constraints: ['shift_Mon1', 'shift_Tue2', 'shift_Wed3', 'shift_Thu4', 'shift_Fri5']


## Solver Configuration and Solution

Configure the solver settings and solve the optimization problem.


In [9]:
# Configure solver settings
settings = SolverSettings()
settings.set_parameter("time_limit", 60.0)  # 60 second time limit
settings.set_parameter("log_to_console", True)  # Enable solver logging
settings.set_parameter("method", 0)  # Use default method

print("Solver configured with 60-second time limit")


Solver configured with 60-second time limit


In [10]:
# Solve the problem
print("Solving workforce optimization problem...")
print(f"Problem type: {'MIP' if problem.IsMIP else 'LP'}")
print(f"Number of variables: {problem.NumVariables}")
print(f"Number of constraints: {problem.NumConstraints}")

problem.solve(settings)

print(f"\nSolve completed in {problem.SolveTime:.3f} seconds")
print(f"Solver status: {problem.Status.name}")
print(f"Objective value: ${problem.ObjValue:.2f}")


Solving workforce optimization problem...
Problem type: MIP
Number of variables: 73
Number of constraints: 14
Setting parameter time_limit to 6.000000e+01
Setting parameter log_to_console to true
Setting parameter method to 0
cuOpt version: 25.10.0, git hash: c426e3a, host arch: x86_64, device archs: 75
CPU: AMD Ryzen Threadripper PRO 3975WX 32-Cores, threads (physical/logical): 32/64, RAM: 20.93 GiB
CUDA 13.0, device: Quadro RTX 8000 (ID 0), VRAM: 47.25 GiB
CUDA device UUID: ffffffb7fffffff2ffffffb679-057e-ffff

Unpresolved problem:: 14 constraints, 73 variables, 73 nonzeros
Presolve status:: reduced the problem
Presolve removed:: 8 constraints, 36 variables, 36 nonzeros
Presolved problem:: 6 constraints, 37 variables, 37 nonzeros
Third party presolve time: 0.119085
Solving a problem with 6 constraints 37 variables (37 integers) and 37 nonzeros
Objective offset 304.000000 scaling_factor 1.000000
Running presolve!
After trivial presolve #constraints 6 #variables 37 objective offset 304

## Solution Analysis

Let's analyze the optimal solution and create visualizations.


In [11]:
def print_solution():
    """Print the optimal solution in a readable format"""
    if problem.Status.name == "Optimal" or problem.Status.name == "FeasibleFound":
        print(f"\nOptimal Solution Found!")
        print(f"Total Labor Cost: ${problem.ObjValue:.2f}")
        print("\nShift Assignments:")
        
        # Group assignments by shift
        shift_assignments = {}
        for (worker, shift), var in assignment_vars.items():
            if var.getValue() > 0.5:  # Binary variable is 1
                if shift not in shift_assignments:
                    shift_assignments[shift] = []
                shift_assignments[shift].append(worker)
        
        # Display assignments by shift
        for shift in sorted(shift_assignments.keys()):
            workers = shift_assignments[shift]
            required = shift_requirements[shift]
            total_cost = sum(worker_pay[w] for w in workers)
            print(f"  {shift}: {workers} (Required: {required}, Assigned: {len(workers)}, Cost: ${total_cost})")
        
        # Display assignments by worker
        print("\nWorker Assignments:")
        worker_assignments = {}
        for (worker, shift), var in assignment_vars.items():
            if var.getValue() > 0.5:
                if worker not in worker_assignments:
                    worker_assignments[worker] = []
                worker_assignments[worker].append(shift)
        
        for worker in sorted(worker_assignments.keys()):
            shifts = worker_assignments[worker]
            total_cost = len(shifts) * worker_pay[worker]
            print(f"  {worker}: {shifts} ({len(shifts)} shifts, ${total_cost})")
            
        return shift_assignments, worker_assignments
    else:
        print(f"No optimal solution found. Status: {problem.Status.name}")
        return None, None

shift_assignments, worker_assignments = print_solution()



Optimal Solution Found!
Total Labor Cost: $468.00

Shift Assignments:
  Fri12: ['Amy', 'Cathy', 'Dan', 'Fred', 'Gu'] (Required: 5, Assigned: 5, Cost: $48)
  Fri5: ['Amy', 'Cathy', 'Dan', 'Ed', 'Gu'] (Required: 5, Assigned: 5, Cost: $47)
  Mon1: ['Ed', 'Fred', 'Gu'] (Required: 3, Assigned: 3, Cost: $28)
  Mon8: ['Dan', 'Ed'] (Required: 2, Assigned: 2, Cost: $16)
  Sat13: ['Amy', 'Bob', 'Cathy', 'Dan', 'Ed', 'Fred', 'Gu'] (Required: 7, Assigned: 7, Cost: $68)
  Sat6: ['Dan', 'Fred', 'Gu'] (Required: 3, Assigned: 3, Cost: $28)
  Sun14: ['Amy', 'Cathy', 'Dan', 'Ed', 'Fred'] (Required: 5, Assigned: 5, Cost: $45)
  Sun7: ['Amy', 'Cathy', 'Ed', 'Gu'] (Required: 4, Assigned: 4, Cost: $39)
  Thu11: ['Amy', 'Cathy', 'Dan', 'Ed'] (Required: 4, Assigned: 4, Cost: $36)
  Thu4: ['Cathy', 'Ed'] (Required: 2, Assigned: 2, Cost: $18)
  Tue2: ['Dan', 'Ed'] (Required: 2, Assigned: 2, Cost: $16)
  Tue9: ['Dan', 'Ed'] (Required: 2, Assigned: 2, Cost: $16)
  Wed10: ['Amy', 'Cathy', 'Dan'] (Required: 3, Ass

In [12]:
# Create a summary table of the solution
if shift_assignments:
    solution_data = []
    for shift in sorted(shift_assignments.keys()):
        workers = shift_assignments[shift]
        required = shift_requirements[shift]
        assigned = len(workers)
        total_cost = sum(worker_pay[w] for w in workers)
        
        solution_data.append({
            'Shift': shift,
            'Required': required,
            'Assigned': assigned,
            'Workers': ', '.join(workers),
            'Cost': f"${total_cost}"
        })
    
    solution_df = pd.DataFrame(solution_data)
    print("\nSolution Summary:")
    print(solution_df.to_string(index=False))



Solution Summary:
Shift  Required  Assigned                            Workers Cost
Fri12         5         5          Amy, Cathy, Dan, Fred, Gu  $48
 Fri5         5         5            Amy, Cathy, Dan, Ed, Gu  $47
 Mon1         3         3                       Ed, Fred, Gu  $28
 Mon8         2         2                            Dan, Ed  $16
Sat13         7         7 Amy, Bob, Cathy, Dan, Ed, Fred, Gu  $68
 Sat6         3         3                      Dan, Fred, Gu  $28
Sun14         5         5          Amy, Cathy, Dan, Ed, Fred  $45
 Sun7         4         4                 Amy, Cathy, Ed, Gu  $39
Thu11         4         4                Amy, Cathy, Dan, Ed  $36
 Thu4         2         2                          Cathy, Ed  $18
 Tue2         2         2                            Dan, Ed  $16
 Tue9         2         2                            Dan, Ed  $16
Wed10         3         3                    Amy, Cathy, Dan  $28
 Wed3         4         4               Cathy, Dan, Ed, F

## Adding Additional Constraints

Now let's demonstrate how to add additional constraints to the existing model. We'll add a constraint to limit the maximum number of shifts per worker.


In [13]:
# Add constraint: each worker can work at most 4 shifts per week
max_shifts_per_worker = 4

for worker in worker_pay.keys():
    # Find all shifts this worker is available for
    worker_shifts = []
    for (w, shift), var in assignment_vars.items():
        if w == worker:
            worker_shifts.append(var)
    
    if worker_shifts:
        # Create constraint: sum of shifts for this worker <= max_shifts_per_worker
        worker_expr = LinearExpression([], [], 0.0)
        for var in worker_shifts:
            worker_expr += var
        
        constraint = problem.addConstraint(worker_expr <= max_shifts_per_worker, 
                                        name=f"max_shifts_{worker}")

print(f"Added maximum shift constraints (max {max_shifts_per_worker} shifts per worker)")


Added maximum shift constraints (max 4 shifts per worker)


In [14]:
# Solve the problem again with the new constraints
print("\nSolving with maximum shift constraints...")
print(f"Problem now has {problem.NumVariables} variables and {problem.NumConstraints} constraints")


problem.solve(settings)

print(f"\nSolve completed in {problem.SolveTime:.3f} seconds")
print(f"Solver status: {problem.Status.name}")
print(f"Objective value: ${problem.ObjValue:.2f}")



Solving with maximum shift constraints...
Problem now has 73 variables and 21 constraints
Setting parameter time_limit to 6.000000e+01
Setting parameter log_to_console to true
Setting parameter method to 0
cuOpt version: 25.10.0, git hash: c426e3a, host arch: x86_64, device archs: 75
CPU: AMD Ryzen Threadripper PRO 3975WX 32-Cores, threads (physical/logical): 32/64, RAM: 21.47 GiB
CUDA 13.0, device: Quadro RTX 8000 (ID 0), VRAM: 47.25 GiB
CUDA device UUID: ffffffb7fffffff2ffffffb679-057e-ffff

Unpresolved problem:: 21 constraints, 73 variables, 146 nonzeros
Presolve status:: found an infeasible problem

Solve completed in 0.000 seconds
Solver status: Infeasible
Objective value: $nan


In [15]:
# Display the new solution
shift_assignments_new, worker_assignments_new = print_solution()


No optimal solution found. Status: Infeasible


## Conclusion

This notebook demonstrated how to:

1. **Formulate a workforce optimization problem** using the cuOpt Python API
2. **Set up binary decision variables** for worker-shift assignments
3. **Define an objective function** to minimize total labor cost
4. **Add shift requirement constraints** to ensure proper staffing
5. **Solve the optimization problem** using cuOpt's high-performance solver
6. **Add additional constraints** to limit worker shifts
7. **Analyze and compare solutions** before and after constraint modifications

The cuOpt Python API provides a clean, intuitive interface for building and solving optimization problems, making it easy to model complex real-world scenarios like workforce scheduling.

### Key Benefits of cuOpt:
- **High Performance**: GPU-accelerated solving for large-scale problems
- **Easy to Use**: Intuitive Python API similar to other optimization libraries
- **Flexible**: Support for both LP and MIP problems
- **Scalable**: Handles problems with thousands of variables and constraints efficiently

### Problem Extensions:
This basic workforce optimization model can be extended with additional constraints such as:
- Minimum rest time between shifts
- Skill requirements for specific shifts
- Overtime cost considerations
- Worker preferences and fairness constraints
- Multi-week scheduling with carryover constraints

## License

SPDX-FileCopyrightText: Copyright (c) 2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
SPDX-License-Identifier: MIT
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
